### Calculate Navigation Analyses Values Per Larvae
#### List of tasks accomplished in this Jupyter Notebook:
- Calculates each of the ten navigation variables for each animal and outputs the results as a separate data file
	1. Time spent moving
	2. Total distance traveled
	3. Mean speed when moving
	4. Maximum speed
	5. Mean speed during the first minute 
	6. Difference in mean speed (first and last minute)
	7. Number of sharp turns (>45 degrees)
	8. Number of spiral movements
	9. Number of continuous paths that are not spirals
	10. Longest rest period (in seconds)

In [1]:
import numpy as np
import pandas as pd
import math, os
import more_itertools as mit

In [2]:
fps = 2
sh = 1 # direction in which to shift vector for delta calculations
threshold = 2 # consecutive seconds spent moving
still_threshold = 10 # seconds for minimum still num count
angle_thresh = 20 # Allowed error from previous angle
sharp_angle = 45
spiral_thresh = 4
spiral_speed_thresh = np.inf

In [3]:
def safe_divide(x, y):
    x = safe_get(x)
    y = safe_get(y)
    try: 
        ans = x / y
    except ZeroDivisionError: 
        return 0
    return ans
    
def safe_get(x, ans=0):
    if math.isnan(x):
        return ans
    return x

In [4]:
# Analyze acclimation data
df = pd.read_csv("./data/experiment_IDs/cleaned_static_data.csv")

animals = df["animal_ID"].unique()
# Check that animal IDs in dataframe are unique
assert len(df) == len(animals)

fps = 2
master_df = pd.DataFrame()

for index, row in df.iterrows():
    animal = row['animal_ID']
    aID = animal[:9]
    pos = animal[10:]
    
    filename = "./data/trajectories/video_calculations/"+aID+'-A-'+pos+'.csv'
    
    if os.path.isfile(filename):
        temp = pd.read_csv(filename)
        
        # Get the change in the change in angle 
        temp['angle_delta_2'] = temp['angle_delta'].shift(sh)
        temp['angle_delta_delta'] = temp['angle_delta']-temp['angle_delta_2']
        temp['angle_delta_delta'] = np.abs(temp['angle_delta_delta'])
        temp_move = temp[temp["moving"] == True]
        
        # % TIME SPENT MOVING, percent ------------------------------
        if len(temp_move) > 0:
            time_move = 100*len(temp_move)/len(temp)
        else:
            time_move = 0
            
        # TOTAL DISTANCE TRAVELED, in meters
        total_dist = temp['speed_mm_s'].sum()/fps/1000
        
        # MAXIMUM SPEED
        max_speed = temp['speed_BL'].max()
        
        # AVERAGE SPEED FOR FIRST MINUTE
        mean_speed_first = temp[temp['frames'] <= 60*fps]['speed_BL'].mean()
        
        # DIFFERENCE BETWEEN FIRST and LAST MINUTES
        diff_speed_f_l = temp[temp['frames'] >= 14*60*fps]['speed_BL'].mean() - mean_speed_first
        
        # AVERAGE SPEED WHEN MOVING
        if len(temp_move) == 0:
            avg_speed = 0
        else:
            avg_speed = temp_move['speed_BL'].mean()
            # CHECKS for any movement of more than a threshold mm/s
            if max(temp_move['speed_mm_s']) > 50:  
                print(max(temp_move['speed_mm_s']), str(aID+'-A-'+pos))
        
        # NUMBER OF SLOW MOVING SPIRALS
        temp_move_side1 = temp_move[temp_move['angle_delta'] < 0]
        temp_move_side2 = temp_move[temp_move['angle_delta'] > 0]
        timestamps1 = [list(x) for x in mit.consecutive_groups(temp_move_side1.index)]
        timestamps2 = [list(x) for x in mit.consecutive_groups(temp_move_side2.index)]
        timestamps1 = [x for x in timestamps1 if len(x) > spiral_thresh*fps]
        timestamps2 = [x for x in timestamps2 if len(x) > spiral_thresh*fps]
        small_spirals, total_timestamps = 0, []
        for thresh_time in timestamps1+timestamps2: 
            thresh_time = [thresh_time[0]-1]+thresh_time
            df_thresh = temp[temp.index.isin(thresh_time)]
            if np.mean(df_thresh['speed_mm_s']) < spiral_speed_thresh:
                small_spirals += 1
                total_timestamps += df_thresh['frames'].tolist()
        
        # NUMBER OF SHARP TURNS
        temp_sharp = temp[temp['speed_mm_s'] > 4]
        temp_sharp = temp_sharp[(temp_sharp['angle_delta'] < -sharp_angle) | \
                                (temp_sharp['angle_delta'] > sharp_angle)]
        temp_sharp = [x-1 for x in temp_sharp['frames'].tolist()]
        temp_sharp = temp[temp['frames'].isin(temp_sharp)]
        sharp_turns = 100*safe_divide(len(temp_sharp),len(temp))
        
        # NUMBER OF CONTINUOUS PATHS
        # PROPORTION OF DISTANCE MOVING CONTINUOUSLY, EXCLUDING SPIRALS
        temp_move_cons = temp_move[temp_move['angle_delta_delta'] <= angle_thresh]
        # Remove spirals using total_timestamps 
        temp_move_cons = temp_move_cons[~temp_move_cons['frames'].isin(total_timestamps)]
        timestamps = [list(x) for x in mit.consecutive_groups(temp_move_cons.index)]
        continuous_nums = 0
        for thresh_time in timestamps: 
            thresh_time = [thresh_time[0]-1]+thresh_time
            df_thresh = temp[temp.index.isin(thresh_time)]
            continuous_nums += 1

        # MAXIMUM REST LENGTH IN SECONDS
        temp_still = temp[temp["moving"] == False]
        timestamps = [list(x) for x in mit.consecutive_groups(temp_still.index)]
        thresh_times = [x for x in timestamps if len(x) > still_threshold*fps]
        still_len = []
        for thresh_time in thresh_times:
            thresh_time = [thresh_time[0]-1]+thresh_time
            df_thresh = temp[temp.index.isin(thresh_time)]
            still_len.append(len(df_thresh))
        if len(still_len) > 0:
            still_len_max = np.max(still_len)/fps # in seconds
        else:
            still_len_max = 0

        temp = pd.DataFrame({
            # STATIC VARIABLES ----------------------------
            "animal_ID": [row["animal_ID"]], 
            "treatment_odor": row["treatment_odor"],
            "sex": row["sex"], 
            "species": row['species'],
            'dead':row['dead'],
            "length_mm": row['larvae_length_mm'],

            # VARIABLES CALCULATED THROUGHOUT VIDEO -------
            "time_move_p": time_move,
            "total_dist_m": total_dist,
            "avg_speed_BL": avg_speed,
            "max_speed_BL": max_speed,
            "mean_speed_first_BL": mean_speed_first,
            "diff_speed_first_last_BL": diff_speed_f_l,
            "sharp_turns_p": sharp_turns,
            "spirals": small_spirals,
            "continuous": continuous_nums, 
            "max_still_sec": still_len_max,
            })
        master_df = pd.concat([master_df, temp])

master_df.to_csv('./data/trajectories/cleaned_animal_analyses_acclimation.csv', index=False)
# display(master_df.head())
print('--- Finished with saving df! ---')

77.68545705420786 190627-02-A-bottom
54.817863352540634 190725-01-A-top
70.02528646997774 190725-01-A-bottom
55.575735572050924 190725-03-A-bottom
62.28632977722879 190725-09-A-top
50.991679543249454 190727-01-A-top
63.107943243087455 190917-06-A-bottom
51.0765108774137 190917-09-A-bottom
56.964707146683395 191106-03-A-top
51.472877920441206 191106-06-A-bottom
56.329090794498406 191107-06-A-top
50.464740966026866 191107-09-A-bottom
102.21385905969385 191109-01-A-bottom
85.14380831235722 191126-01-A-bottom
--- Finished with saving df! ---
